In [ ]:
# Initialization cell
try:  # for CS1302 JupyterLite pyodide kernel
    import piplite

    with open("requirements.txt") as f:
        for package in f:
            package = package.strip()
            print("Installing", package)
            await piplite.install(package)
except ModuleNotFoundError:
    pass

import random
import jupytext
import otter
from ipywidgets import interact

grader = otter.Notebook("main.ipynb")
%reload_ext divewidgets

# Combinatorics

**CS1302 Introduction to Computer Programming**
___

## Permutation using recursion

---

**Definition**

A [$k$-permutation of $n$](https://en.wikipedia.org/wiki/Permutation#k-permutations_of_n) items $a_0,\dots,a_{n-1}$ is an ordered tuple 

$$
(a_{i_0},\dots,a_{i_{k-1}})
$$ 
of $k$ out of the $n$ objects, where $0\leq i_0,\dots,i_{k-1}<n$ are distinct indices. An $n$-permutation of $n$ objects is called a permutation of $n$ objects.

---

For examples:
- The list of ($3$-)permutations of `1,2,3` is:

In [ ]:
[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

- The list of $2$-permutations of `1,2,3` is:

In [ ]:
[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]

In the above, we used
- a `tuple` delimited by `()` such as `(1,2,3)` to store items of a permutation, and 
- a `list` delimited by `[]` such as `[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]` to store all the permutations.

Generating permutations is a fundamental problem in computing and combinatorics.  
A simple way to generate permutations is by recursion. (There are also [other algorithms](https://www.topcoder.com/generating-permutations/).)

---

**Proposition**

A $k$-permutation of $n$ satisfies the recurrence relation:  
- Removing the first element of a $k$-permutation of $n$ objects gives a different $(k-1)$-permutation of the remaining $n-1$ objects.

$$ (a_{i_0}, \underbrace{a_{i_1},\dots,a_{i_{k-1}}}_{\text{($k-1$)-permutation of $a_{i_1},\dots,a_{i_{k-1}}$.}\kern-5em} ) $$

- Reversing the above removal process gives a way of constructing a $k$-permutation from a $(k-1)$-permutation.  
  E.g., the permutations of $1,2,3$ can be constructed as follows:

$$[\overbrace{({\color{red} 1}, {\color{blue}{2, 3}}), ({\color{red} 1}, {\color{blue}{3, 2}})}^{\text{prepend 1 to permutations of $2,3$.} }, \overbrace{({\color{red} 2}, {\color{blue}{1, 3}}), ({\color{red} 2}, {\color{blue}{3, 1}})}^{\text{prepend 2 to permutations of $1,3$.} }, \overbrace{({\color{red} 3}, {\color{blue}{1, 2}}), ({\color{red} 3}, {\color{blue}{2, 1}})}^{\text{prepend 3 to permutations of $1,2$.} }]$$

---

The following implements the recursion:

In [ ]:
def permutation(*a, k=None):
    """Construct all (k-)permutations of the position arguments.

    Parameters
    ----------
    *a: object, object, ...
        n items specified as positional arguments
    k: int
        Optional argument indicating the size of each permutation. 
        Default: number n of items specified in *a.

    Returns
    -------
    list:
        List of all k-permutations represented as ordered tuples of the n objects.
    """
    n = len(a)
    output = []
    if k is None:
        k = n
    if 0 < k <= n:
        for i in range(n):
            output.extend(
                [(a[i],) + p for p in permutation(*a[:i], *a[i + 1 :], k=k - 1)]
            )
    elif k == 0:
        return [()]
    return output


print(permutation(1, 2, 3))
print(permutation(1, 2, 3, k=2))

The recurrence is implemented by the for loop:
```Python
        ...
        for i in range(n):
            output.extend([(a[i], ) + p
                           for p in permutation(*a[:i], *a[i + 1:], k=k - 1)])
        ...
```

In the above code, `(a[i],) + p` creates a $k$-permutation of the items in `a` by concatenating for each index `i`
- a singleton tuple `(a[i],)` and 
- a $(k-1)$-permutation `p` of all elements but `a[i]`.

(See the example in the recurrence relation described earlier.)

```{note}

- The comma in `(a[i],)` is not a typo. Without commas, `(...)` does not create a tuple. 
- `a[:i]` returns a tuple of `a[0]` up to and excluding `a[i]`. `*a[:i]` unpacks the tuple such that its items are separate arguments to `permutation`. 
- Similarly, `*a[i + 1:]` provides items as separate arguments starting from `a[i + 1]` until the last item in `a`.
- `[... for ...]` generates a list of $k$-permutations, and `output.extend([...])` added the list to the end of the `output` list.

```

**Exercise** (base)

One of the base cases of the recursion happens when $k=0$, in which case there is only one $k$-permutation, namely the empty tuple $()$, and so the function returns `[()]`. There is another base case of the recursion. Explain the condition of that base case and its return value.

_Type your answer here, replacing this text._

## Number of permutations

Computing permutations using recursion is slow. Why?  

There are too many permutations, even for a reasonably small $n$. Try running the following code in a new console:

```python
n = 10
output = permutation(*range(1, n + 1))
print(f"# {n}-permutations:", len(output))
```

```{caution}
Avoid creating a new cell to run the above code because auto-grader may refuse to execute code that takes too long to run. To open a new console, right-click anywhere and select `New Console for Notebook`.
```

Quite surprisingly, the number of permutations can be calculated significantly faster without enumerating all the permutations.

---

**Proposition**

The number $P_n$ of ($n-$)permutations of $n$ items satisfies the following recurrence:

$$
P_n = \begin{cases}
n P_{n-1} & n>0\\
1 & n=0\\
0 & \text{otherwise.}
\end{cases}
$$ (Pn)

---

The quantity is fundamental in the field of [combinatorics](https://en.wikipedia.org/wiki/Combinatorics) with enormous applications.

**Exercise** (num-permutation)

Implement the above recurrence equation.

```{hint}

Ensure all base cases are covered, and the code can run efficiently for large $n$.

```

In [ ]:
def num_permutation_v1(n):
    """Compute the number of permutations.

    Parameters
    ----------
    n: int
        Number of items.

    Return
    ------
    int:
        Number of permutations of n items.
    """
    ...

In [ ]:
grader.check("num-permutation")

---

**Proposition**

The number $P_{n,k}$ of $k$-permutations of $n$ items is given by the formula

$$
P_{n,k} = \begin{cases}
\frac{P_n}{P_{n-k}} & 0\leq k \leq n\\
0 & \text{otherwise.}
\end{cases}
$$ (Pnk)

---

**Exercise** (optional-argument)

Extend the function `num_permutation_v1` to `num_permutation_v2` to allow for a optional argument `k`.

In [ ]:
def num_permutation_v2(n, k=None):
    """Compute the number of k-permutations of n items.

    Parameters
    ----------
    n: int
        Number of items to permute.
    k: int
        Optional argument indicating the size of each permutation.
        Default: n

    Returns
    -------
    int:
        Number of k-permutations of n items.
    """
    ...

In [ ]:
grader.check("optional-argument")

## Permutation using iteration

The following function `permutation_sequence(*a)` returns a generator that generates the list of $k$-permutations one-by-one for $k$ from $0$ to `len(a)`.

In [ ]:
def permutation_sequence(*a):
    """Returns a generator for the k-permutations of the positional arguments
    for k from 0 to len(a)."""
    n = len(a)
    output, idx_left = [()], [tuple(range(n))]
    for k in range(n + 1):
        yield output
        next_output, next_idx_left = [], []
        for m in range(len(idx_left)):
            for j in range(len(idx_left[m])):
                i = idx_left[m][j]
                next_output.append(output[m] + (a[i],))
                next_idx_left.append(idx_left[m][:j] + idx_left[m][j + 1 :])
        output, idx_left = next_output, next_idx_left


for permutation_list in permutation_sequence(1, 2, 3):
    print(permutation_list)

Unlike the recursive function `permutation`, the above generates a $k$-permutation $(a_{i_0},\dots,a_{i_{k-1}})$ of $n$ items iteratively by 
- choosing $i_j$ for $j$ from $0$ to $k-1$ such that
- $i_j$ is not already chosen, i.e., $i_j\not\in \{i_0,\dots,i_{j-1}\}$.

E.g., the permutations of $1,2,3$ is generated iteratively as follows:
- 1
 - 1,2
   - **(1,2,3)**
 - 1,3
   - **(1,3,2)**
- 2
 - 2,1
   - **(2,1,3)**
 - 2,3
   - **(2,3,1)**
- 3
 - 3,1
   - **(3,1,2)**
 - 3,2
   - **(3,2,1)**

---

**Proposition**

`permutation_sequence` maintains the following *invariance* at the beginning of each iteration:  
- `output` is the list of $k$-permutations, and
- `idx_left[m]` is the list of indices of arguments not yet in `output[m]`. 

A $(k+1)$-permutation (in `next_output`) can then be generated by appending an argument (with an index from `idx_left`) to a $k$-permutation (in `output`).

---

Is iteration significantly faster?

Try running the following code in a new console:

```python
n = 10
for k, permutation_list in enumerate(permutation_sequence(*range(1, n + 1))):
    print("# {}-permutations of {} items: {}".format(k, n, len(permutation_list)))
```

Unfortunately, there is not much improvement. Nevertheless, we can efficiently compute the number of $k$-permutations based on the previously computed number of $k-1$-permutations:

For $k$ from $0$ to $n$,

$$
P_{n,k} = \underbrace{\overbrace{n\times  (n-1)\times \cdots }^{P_{n,k-1}\text{  if }k>0}\times(n-k+1)}_{\text{$k$ terms in the product.}}.$$ (Pnk:expanded)

**Exercise** (yield)

Use the `yield` statement to write the function `num_permutation_sequence_v1(n)` that returns a generator of $P_{n,k}$ with `k` from `0` to `n`.

In [ ]:
def num_permutation_sequence_v1(n):
    output = 1
    for k in range(0, n + 1):
        ...

In [ ]:
grader.check("yield")

**Exercise** (send)

Extend the function `num_permutation_sequence_v1(n)` to `num_permutation_sequence_v2(n)` so that calling `send(0)` method causes the generator to increment $n$ instead of $k$ for the next number to generate. i.e., for $0\leq k \leq n$,

$$\dots P_{n,k-1}\to P_{n,k} \xrightarrow{\text{send(0)}} P_{n+1,k} \to P_{n+1,k+1}\dots$$
where $\to$ without labels is the normal transition without calling the `send` method.

```{hint}

By {eq}`Pnk:expanded`,

$$P_{n+1,k}=P_{n,k} \times \frac{n+1}{n-k+1}.$$

```

In [ ]:
def num_permutation_sequence_v2(n):
    ...

In [ ]:
grader.check("send")

## Deduplication using Decorator

An issue with the function `permutation` is that it regards arguments at different positions as distinct, even if they may have the same value. E.g.,  
`permutation(1,1,2)` returns `[(1, 1, 2), (1, 2, 1), (1, 1, 2), (1, 2, 1), (2, 1, 1), (2, 1, 1)]`  
where each distinct permutation appears twice.

To remove duplicates from a list, we can  
- convert the list to a `set` (which automatically removes duplicates),
- and then convert the set back to a list.

In [ ]:
print("Deduplicated:", list(set(permutation(1, 1, 2))))

Using a decorator, we can fix `permutation` without rewriting the function.

In [ ]:
import functools


def deduplicate_output(f):
    """Takes in a function f that returns a list possibly with duplicates,
    returns a decorator that removes duplicates from the output list."""

    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        return list(set(f(*args, **kwargs)))

    return wrapper


permutation = deduplicate_output(permutation)
print("Deduplicated: ", permutation(1, 1, 2))
permutation = permutation.__wrapped__
print("Original: ", permutation(1, 1, 2))

**Exercise** (deduplicate-input)

Create a decorator to eliminate duplicate input positional arguments instead of the output, i.e.,  
`permutation(1,1,2)` will return the same result as `permutation(1,2)`.

In [ ]:
def deduplicate_input(f):
    """Takes in a function f that takes a variable number of arguments
    possibly with duplicates, returns a decorator that removes duplicates
    in the positional argument."""

    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        ...

    return wrapper

In [ ]:
grader.check("deduplicate-input")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# extra files to submit
extra_files = []

In [ ]:
# Generate the source main.py necessary for grading and similarity check.
jupytext.write(jupytext.read("main.ipynb"), "main.py", fmt="py:percent")

In [ ]:
# Generate the zip file to submit.
grader.export(pdf=False, run_tests=False, files=["main.py", *extra_files])